In [4]:
# Database used: https://www.kaggle.com/datasets/puneet6060/intel-image-classification/data
# TODO Maybe we should not be using pictures of glaciers, mountains, etc., just one single type of image
#  (for now)

import os
import tensorflow as tf
from matplotlib import pyplot as plt
import cv2
import numpy as np

2023-12-24 15:48:57.234432: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-24 15:48:57.267036: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-24 15:48:57.267057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-24 15:48:57.268078: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-24 15:48:57.273073: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-24 15:48:57.273720: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
def load(image_path):
    sample_image = tf.io.read_file(image_path)
    sample_image = tf.io.decode_jpeg(sample_image)
    print(sample_image.shape)
    plt.figure()
    plt.imshow(sample_image)

In [6]:
# TODO [vukan] : This is just for testing, will update the path!
TRAIN_PATH = "../dataset/seg_train/seg_train/mountain"

In [7]:

def go_though_database(database_path):
    all_image_paths = []
    for filename in os.listdir(database_path):
        f = os.path.join(database_path, filename)
        # checking if it is a file
        if os.path.isfile(f):
            all_image_paths.append(f)
            #print(f)
            #load(f)
            #return
    return all_image_paths

In [8]:
all_image_paths = go_though_database(TRAIN_PATH)

In [9]:
def create_mask(image):
    img_height, img_width = image.shape[:2] #(150, 150, 3)
    mask = 255 * np.ones_like(image) # matrix of ones -> white image
    blackout_start_index = img_width // 3
    
    for i in range(img_height):
        for j in range(img_width):
            if j >= blackout_start_index and j < 2*blackout_start_index:
                mask[i, j] = 0

    return mask

In [10]:
# applys mask to the image 
def edit_image(image_path):
    image = cv2.imread(image_path);
    mask = create_mask(image)

    #print(image.shape)
    #print(mask.shape)
    
    edited_image = cv2.bitwise_and(image, mask)
    
    #cv2.imshow('mask', mask)
    #cv2.imshow('image', image)
    #cv2.imshow('edited_image', edited_image)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    return edited_image


In [11]:
#edit_image('../dataset/seg_train/seg_train/mountain/9311.jpg')

In [14]:
#TODO: maybe to add categories or something
def save_edited_images(image_paths):
    directory = "../dataset/edited/mountain" #use here name of a given category
    if not os.path.exists(directory):
            os.makedirs(directory)
    print(image_paths)
    for image_path in image_paths:
        image = edit_image(image_path)
        edited_image_name = image_path.split('/')[-1].split('.')[0] + '_edited.jpg'
        edited_image_path = os.path.join(directory, edited_image_name)
        cv2.imwrite(edited_image_path, image)
        

In [13]:
save_edited_images(all_image_paths)

['../dataset/seg_train/seg_train/mountain/9311.jpg', '../dataset/seg_train/seg_train/mountain/16180.jpg', '../dataset/seg_train/seg_train/mountain/3460.jpg', '../dataset/seg_train/seg_train/mountain/12311.jpg', '../dataset/seg_train/seg_train/mountain/15448.jpg', '../dataset/seg_train/seg_train/mountain/2930.jpg', '../dataset/seg_train/seg_train/mountain/19207.jpg', '../dataset/seg_train/seg_train/mountain/293.jpg', '../dataset/seg_train/seg_train/mountain/10922.jpg', '../dataset/seg_train/seg_train/mountain/10058.jpg', '../dataset/seg_train/seg_train/mountain/15510.jpg', '../dataset/seg_train/seg_train/mountain/7015.jpg', '../dataset/seg_train/seg_train/mountain/18578.jpg', '../dataset/seg_train/seg_train/mountain/18726.jpg', '../dataset/seg_train/seg_train/mountain/12105.jpg', '../dataset/seg_train/seg_train/mountain/2687.jpg', '../dataset/seg_train/seg_train/mountain/1160.jpg', '../dataset/seg_train/seg_train/mountain/16128.jpg', '../dataset/seg_train/seg_train/mountain/19791.jpg', 